In [99]:
import pathlib, pickle

def create_strat(alpha, Z):
    #we consider only Zero Determinant Strategies so 
    #-1/P <= alpha + beta <= -1/R
    #alpha >= -1
    #alpha + beta = -1/Z for P <= Z <= R
    assert P <= Z <= R
    assert alpha >= -1
    return (alpha, -1 * (alpha + 1/Z))

#calculate payoffs sx, sy when player X with strat1 plays player Y with strat2
def calculate_payoff(strat1, strat2): 
    alpha, beta, a, b = strat1[0], strat1[1], strat2[0], strat2[1]
    D = beta * b - alpha * a #Determinant
    sx = 1/D * (a - beta) 
    sy = 1/D * (alpha - b)
    return (sx, sy)

def create_game_matrix(strats): 
    A = matrix(RR, 3, 3) #initialize an identity matrix
    for i in range(len(strats)): 
        for j in range(i, len(strats)): 
            sx, sy = calculate_payoff(strats[i], strats[j])
            # print(i, j, sx, sy)
            A[i, j] = sx
            A[j, i] = sy
    return A


def compute_game(strats, pi, A):
    t = var('t') 
    pi_var = vector(var("pi_0, pi_1, pi_2"))
    
    A_i_pi = list()
    for i in range(3): 
        A_i_pi.append(pi_var.dot_product(A[i]))

    A_pi_pi = pi_var.dot_product(vector(A_i_pi))

    de_system = list()

    for i in range(3):
        de_system.append( pi_var[i] * (A_i_pi[i] - A_pi_pi) )
        
    
    soln = desolve_system_rk4(de_system, pi_var, ivar = t, ics = [0] + pi, step = .2, end_points=100)
    #2D list with each 1D list of form [t, de1, de2, de3, ... deN]

    return soln

def generate_graphs(strats, pi, game_matrix, folder): 

    soln = compute_game(strats, pi, game_matrix)

    strat0 = [[i,j] for i,j,k,l in soln]
    strat1 = [[i,k] for i,j,k,l in soln]
    strat2 = [[i,l] for i,j,k,l in soln]
    flow = [[j,k] for i,j,k,l in soln]

    #create time plot
    legend = lambda num, strat: "strat " + str(num) + ": " + str(strat)
    LP1 = list_plot(strat0, color = "blue", legend_label= legend(0, strats[0]))
    LP2 = list_plot(strat1, color = "purple", legend_label= legend(1, strats[1]))
    LP3 = list_plot(strat2, color = "green", legend_label= legend(2, strats[2]))
    Time_plot = LP1 + LP2 + LP3
    Time_plot.axes_labels(["$t$", "population ratio"])
    Time_plot.show(title = "pop. initial cond: " + str(pi))

    #create flow plot
    Flow = list_plot(flow, color="red") + points([flow[0]], color="blue", size = "40", legend_label="start") + points([flow[-1]], color="green", size = "40", legend_label="end")
    Flow_plot = Flow + plot(1 - x, 0, 1, color = "grey")
    Flow_plot.axes_labels([r"$\pi_0$", r"$\pi_1$"])
    Flow_plot.show(title = r"initial cond: [$\pi_0, \pi_1, \pi_2$] = " + str(pi))
    
    #save plots
    Time_plot.save(str(folder) + "/time.png")
    Flow_plot.save(str(folder) + "/flow.png") 


    

In [102]:
S, P, R, T = 1, 2, 3, 4

assert S < P < R < T
assert 2 * R > T + S

for it in range(10):
    parameters = [ (uniform(-1, 1), uniform(P, R)) for i in range(3)]

    strats = [ create_strat(*parm) for parm in parameters ]

    populations = [RR.random_element(0, 100) for i in range(3)] 

    total = sum(populations)
    pi = [pop/total for pop in populations] #initial conditions

    A = create_game_matrix(strats)

    #create folder
    # folder_name = "IC: " + str(pi) + ";strats: " + str(strats)
    folder_name = f"example_{it}"
    folder = pathlib.Path(folder_name)
    folder = folder.resolve()
    folder.mkdir()


    #save parameters as txt file
    with open(str(folder) + "/parameters.txt", 'w') as f: 
        f.write(f'S: {S}, P: {P}, R: {R}, T:{T}\n')
        f.write("pi aka initial conditions of population: " + str(pi) + "\n") 
        f.write("strategies in format (alpha, beta): " + str(strats) + "\n")

    #pickle pi and strats 
    with open(str(folder) + "/pi_pickle", 'wb') as f: 
        pickle.dump(pi, f)

    with open(str(folder) + "/strats_pickle", 'wb') as f:
        pickle.dump(strats, f)

    generate_graphs(strats, pi, A, folder)

FileExistsError: [Errno 17] File exists: '/Users/tguan/Documents/akin_independent_study/example_0'